# Strictness and Laziness
**Non-strictness** is a fundamental technique for improving on
- efficiency
- modularrity

of functional programs in general.

## Motivating Example
```scala
List(1,2,3,4).map(_ + 10).filter(_ % 2 == 0).map(_ * 3)
```

## Clear Intent does not mean... Familiar!
Find the square of the second even number which is greater than 7.

```scala
// inparative style
Integer find(List<Integer> ints) {
  int count = 0; int ans = 0;
  for(Integer num : ints) {
    if(num % 2 == 0) {
      if(num > 7) {
        count++;
        if(count == 2) {
          ans = num * num; break;
        }
      }
    }
  }
  return ans;
}
```

```scala
// functional style
Optional<Integer> find(List<Integer> ints) {
  return(
    ints.stream()
        .filter(n -> n % 2 == 0)
        .filter(n -> n > 7)
        .skip(1)
        .map(n -> n * n)
        .findFirst()
  );
}
```

# stream

In [1]:
sealed trait Stream[+A] {
  def foldRight[B](z: => B)(f: (A, => B) => B): B = this match {
    case Cons(h, t) => f(h(), t().foldRight(z)(f))
    case _ => z
  }
  
  def exists_(p: A => Boolean): Boolean = 
    foldRight(false)((a, acc) => p(a) || acc)
  
  def forAll(p: A => Boolean): Boolean = 
    foldRight(true)((a, acc) => p(a) && acc)
  
  def headOption_foldRight: Option[A] =
    foldRight(None: Option[A])((a, acc) => Some(a))
  
  def headOption: Option[A] = this match {
    case Empty => None
    case Cons(h, t) => Option(h())
  }
  
//   def map[B](f: A => B): Stream[B] =
//     foldRight(Empty)((a, acc) => cons(f(a), acc))

//   def mapViaFlatMap[B](f: A => B): Stream[B] =
//     flatMap( a=> Stream(f(a)))
  // variant error version
//   def append(b: => Stream[A]): Stream[A] =
//     foldRight(b)((a, acc) => cons(a, acc))
  
//   def append[B >: A](b: => Stream[B]): Stream[B] =
//     foldRight(b)(cons(_,_))
  
//   def flatMap[B](f: A => Stream[B]): Stream[B] =
//     foldRight(empty[B])((a, acc) => f(a).append(acc))
  
//   def takeWhileWithFoldRIght(P: A => Boolean): Stream[A] =
//     foldRight(empty[A])((a, acc) => if (p(a)) cons(a, acc) else empty[A])
}

case object Empty extends Stream[Nothing]
case class Cons[+A](h: () => A, t: () => Stream[A]) extends Stream[A]

object Stream {
  def cons[A](hd: => A, tl: => Stream[A]): Stream[A] = {
    lazy val head = hd  // We cache the head and tail as lazy values
    lazy val tail = tl  // to avoid repeated evaluation
    Cons(() => head, () => tail)
  }
  
  def empty[A]: Stream[A] = Empty
  
  def apply[A](as: A*): Stream[A] =
    if(as.isEmpty) empty else cons(as.head, apply(as.tail: _*))
  
// //   def take
// //   def takeWhile
// //   def drop
//   def exists(p: A => Boolean): Boolean = this match {
//     case Cons(h, t) => p(h()) || {println("process tail ..."); t().exsits(p)}
//     case _ => false
//   }
}

defined trait Stream
defined object Empty
defined class Cons
defined object Stream

## infinite stream

In [1]:
val ones: Stream[Int] = cons(1, ones)

cmd1.sc:1: not found: value cons
val ones: Stream[Int] = cons(1, ones)
                        ^

: 

In [3]:
ones.take(5)

cmd3.sc:1: value take is not a member of cmd3Wrapper.this.cmd2.cmd0.wrapper.Stream[Int]
val res3 = ones.take(5)
                ^

: 